Прочтём данные из файлов с результатами A/B-тестирования:

In [1]:
import pandas as pd

In [2]:
orders = pd.read_csv('datasets/orders.csv', parse_dates=['date'])

In [3]:
visitors = pd.read_csv('datasets/visitors.csv', parse_dates=['date'])

In [4]:
orders

,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,1650,B
1,2804400009,3642806036,2019-08-15,730,B
2,2961555356,4069496402,2019-08-15,400,A
3,3797467345,1196621759,2019-08-15,9759,B
4,2282983706,2322279887,2019-08-15,2308,B
...,...,...,...,...,...
1192,2662137336,3733762160,2019-08-14,6490,B
1193,2203539145,370388673,2019-08-14,3190,A
1194,1807773912,573423106,2019-08-14,10550,A
1195,1947021204,1614305549,2019-08-14,100,A


In [5]:
visitors

,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756
...,...,...,...
57,2019-08-27,B,720
58,2019-08-28,B,654
59,2019-08-29,B,531
60,2019-08-30,B,490


In [24]:
visitors_a = visitors.query('group == "A"').groupby(['date'], as_index=False).agg({'date' : 'max',
                                                                     'group' : 'max',
                                                                     'visitors' : 'sum'}, axis=1)
visitors_a['visitors'] = visitors_a['visitors'].cumsum()

In [25]:
visitors_a

,date,group,visitors
0,2019-08-01,A,719
1,2019-08-02,A,1338
2,2019-08-03,A,1845
3,2019-08-04,A,2562
4,2019-08-05,A,3318
5,2019-08-06,A,3985
6,2019-08-07,A,4629
7,2019-08-08,A,5239
8,2019-08-09,A,5856
9,2019-08-10,A,6262


In [ ]:
orders_aggregated = orders.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                                          'group' : 'max',
                                                                          'transactionId' : 'nunique',
                                                                          'visitorId' : 'nunique',
                                                                          'revenue' : 'sum'}, axis=1)

In [ ]:
visitors_a

In [26]:
visitors_a_aggregated = visitors.query('group == "A"').groupby(['date'], as_index=False).agg({'date' : 'max',
                                                                                             'group' : 'max',
                                                                                             'visitors' : 'sum'}, axis=1)

In [ ]:
def cumulate_column(df, column):
    grouped_by_a = df[df['group'] == 'A'][column].cumsum()
    grouped_by_b = df[df['group'] == 'B'][column].cumsum()
    cumulated = pd.concat([grouped_by_a, grouped_by_b]).reset_index()
    cumulated.sort_values(by='index', inplace=True)
    return cumulated.set_index('index')

In [ ]:
visitors_aggregated = visitors.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                                              'group' : 'max',
                                                                              'visitors' : 'sum'}, axis=1)

In [ ]:
orders_aggregated = orders.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                                          'group' : 'max',
                                                                          'transactionId' : 'nunique',
                                                                          'visitorId' : 'nunique',
                                                                          'revenue' : 'sum'}, axis=1)

In [ ]:
visitors_aggregated

In [ ]:
orders_aggregated

In [ ]:
cumulate_column(visitors_aggregated, 'visitors')

In [ ]:
cumulate_column(orders_aggregated, 'revenue')

In [ ]:
visitors_aggregated['visitors'] = cumulate_column(visitors_aggregated, 'visitors')
orders_aggregated['transactionId'] = cumulate_column(orders_aggregated, 'transactionId')
orders_aggregated['visitorId'] = cumulate_column(orders_aggregated, 'visitorId')

In [ ]:
visitors_aggregated

In [ ]:
orders_aggregated

In [ ]:
cumulativeData = orders_aggregated.merge(visitors_aggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

cumulativeData.head(11)

In [ ]:
cumulativeData = orders_aggregated.merge(visitors_aggregated)
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

cumulativeData

In [ ]:
visitors[visitors['date'] == '2019-08-05']

In [ ]:
orders[orders['date'] == '2019-08-05'].count()

In [ ]:
temp = orders[orders['date'] == '2019-08-05']

In [ ]:
temp.groupby(['date'], as_index=False).agg({'date' : 'max',
                                            'group' : 'max',
                                            'transactionId' : 'nunique',
                                            'visitorId' : 'nunique',
                                            'revenue' : 'sum'}, axis=1)

In [ ]:
import matplotlib.pyplot as plt

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А
cumulativeRevenueA = cumulativeData[cumulativeData['group']=='A'][['date','revenue', 'orders']]

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B
cumulativeRevenueB = cumulativeData[cumulativeData['group']=='B'][['date','revenue', 'orders']]

# Строим график выручки группы А
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue'], label='A')

# Строим график выручки группы B
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue'], label='B')

plt.legend()

In [ ]:
from solver.ab_reporter import ABReporter

In [ ]:
example = ABReporter('datasets/visitors.csv', 'datasets/orders.csv')

In [ ]:
cumulated = example.grouped_summary()

In [ ]:
cumulated.info()

In [ ]:
    cumulated_copy = cumulated.copy()
    # cumulated_copy['date'] = cumulated_copy['date'].dt.date

In [ ]:
cumulated_copy['date'] = cumulated_copy['date'].dt.date

In [ ]:
cumulated_copy.info()

In [ ]:
def plot_cumulative_metrics():
    """Функция для визуализации кумулятивных метрик."""
    plt.figure(figsize=(25, 10))
    plt.style.use('seaborn-darkgrid')
    
    columns_to_pick = ['date','revenue', 'orders', 'conversion']
    cumulated_copy = cumulated.copy()
    cumulated_copy['date'] = cumulated_copy['date'].dt.date
    revenue_a = cumulated_copy.query('group == "A"')[columns_to_pick]
    revenue_b = cumulated_copy.query('group == "B"')[columns_to_pick]
    merged_revenues = revenue_a.merge(revenue_b, left_on='date', right_on='date', how='left', suffixes=['A', 'B'])
    

    # кривые удержания платящих пользователей
    ax1 = plt.subplot(2, 3, 1)
    plt.plot(revenue_a['date'], revenue_a['revenue'], label='группа A')
    plt.plot(revenue_b['date'], revenue_b['revenue'], label='группа B')
    ax1.set_xticks(revenue_a['date'][::7])
    ax1.set_xticklabels(revenue_a['date'][::7])
    plt.legend()
    plt.ylabel('Выручка')
    plt.xlabel('Лайфтайм')
    plt.title('Графики кумулятивной выручки по дням и группам')
    
    ax2 = plt.subplot(2, 3, 2, sharex=ax1)    
    plt.plot(revenue_a['date'], revenue_a['revenue']/revenue_a['orders'], label='A')
    plt.plot(revenue_b['date'], revenue_b['revenue']/revenue_b['orders'], label='B')
    plt.legend()
    plt.ylabel('Средняя сумма чека')
    plt.xlabel('Лайфтайм')
    plt.title('Графики среднего чека по группам')
              
    ax3 = plt.subplot(2, 3, 3, sharex=ax1)   
    plt.plot(merged_revenues['date'], (merged_revenues['revenueB']/merged_revenues['ordersB'])/(merged_revenues['revenueA']/merged_revenues['ordersA'])-1)
    plt.axhline(y=0, color='black', linestyle='--')
    plt.ylabel('Отношение средних чеков')
    plt.xlabel('Лайфтайм')
    plt.title('График относительного различия для среднего чека')


    ax4 = plt.subplot(2, 3, 4, sharex=ax1)
    plt.plot(revenue_a['date'], revenue_a['conversion'], label='A')
    plt.plot(revenue_b['date'], revenue_b['conversion'], label='B')
    plt.legend()
    plt.ylabel('Конверсия')
    plt.xlabel('Лайфтайм')
    plt.title('График кумулятивной конверсии')
    
    ax5 = plt.subplot(2, 3, 5, sharex=ax1)
    plt.plot(merged_revenues['date'], merged_revenues['conversionB']/merged_revenues['conversionA']-1)
    plt.axhline(y=0, color='black', linestyle='--')
    plt.axhline(y=-0.1, color='grey', linestyle='--')
    plt.title('Относительный прирост конверсии группы B относительно группы A')
    plt.tight_layout()
    plt.show()

In [ ]:
plot_cumulative_metrics()

In [ ]:
cumulated.groupby(['group', 'date'])['revenue'].sum().plot()

In [ ]:
example.plot_cumulative_metrics()

In [ ]:
columns_to_pick = ['date','revenue', 'orders']
conversion_hist = cumulated[columns_to_pick]

In [ ]:
cumulated

In [ ]:
cumulated[['revenue']].plot()

In [ ]:
columns = [name for name in conversion_hist.index.names
           if name not in ['date']]
filtered_data = conversion_hist.pivot_table(
    index='date')

In [ ]:
filtered_data

Аналогично получим агрегированные кумулятивные по дням данные о посетителях интернет-магазина:


In [ ]:
df = visitors.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                        'group' : 'max',
                                                        'visitors' : 'sum'}, axis=1).sort_values(by=['date','group'])

In [ ]:
# создаем массив уникальных пар значений дат и групп теста
datesGroups = orders[['date','group']].drop_duplicates()

# получаем агрегированные кумулятивные по дням данные о заказах 
ordersAggregated = datesGroups.apply(lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'transactionId' : 'nunique', 'visitorId' : 'nunique', 'revenue' : 'sum'}), axis=1).sort_values(by=['date','group'])

# получаем агрегированные кумулятивные по дням данные о посетителях интернет-магазина 
visitorsAggregated = datesGroups.apply(lambda x: visitors[np.logical_and(visitors['date'] <= x['date'], visitors['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'visitors' : 'sum'}), axis=1).sort_values(by=['date','group'])

# объединяем кумулятивные данные в одной таблице и присваиваем ее столбцам понятные названия
cumulativeData = ordersAggregated.merge(visitorsAggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

print(cumulativeData.head(5))